In [1]:
#package loading 
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lifelines import CoxPHFitter
from dateutil.relativedelta import relativedelta
from scipy.stats import percentileofscore
import umap
from tqdm import tqdm
import cudf
import pickle

In [2]:
from utility import *
from preprocess import *

In [3]:
import warnings
warnings.filterwarnings("ignore")

# extract cancer paptients

In [6]:
all_cancer_pid = pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/pid_dx_cancer_subtypes.pkl','rb'))

In [7]:
data_wd = "/data/datasets/Tianchen/2022_covid_pasc_1FL_data/20221020/covid_database"

In [8]:
# covid database
death_cov = pd.read_csv(data_wd+"/DEATH.csv",header=0,skiprows=[1])
diagnosis_cov = pd.read_csv(data_wd+"/DIAGNOSIS.csv",header=0,skiprows=[1])
encounter_cov = pd.read_csv(data_wd+"/ENCOUNTER.csv",header=0,skiprows=[1])
lab_result_cm_cov = pd.read_csv(data_wd+"/LAB_RESULT_CM.csv",header=0,skiprows=[1])
med_admin_cov = pd.read_csv(data_wd+"/MED_ADMIN.csv",header=0,skiprows=[1])
obs_gen_cov = pd.read_csv(data_wd+"/OBS_GEN.csv",header=0,skiprows=[1])
procedure_cov = pd.read_csv(data_wd+"/PROCEDURES.csv",header=0,skiprows=[1])
demographics_cov = pd.read_csv(data_wd+"/DEMOGRAPHIC.csv",header=0,skiprows=[1])
dispensing_cov = pd.read_csv(data_wd+"/DISPENSING.csv",header=0,skiprows=[1])
immunization_cov = pd.read_csv(data_wd+"/IMMUNIZATION.csv",header=0,skiprows=[1])
lds_address_history_cov = pd.read_csv(data_wd+"/LDS_ADDRESS_HISTORY.csv",header=0,skiprows=[1])
obs_CLIN_cov = pd.read_csv(data_wd+"/OBS_CLIN.csv",header=0,skiprows=[1])
prescribing_cov = pd.read_csv(data_wd+"/PRESCRIBING.csv",header=0,skiprows=[1])
vital_cov = pd.read_csv(data_wd+"/VITAL.csv",header=0,skiprows=[1])

In [9]:
data_wd = "/data/datasets/Tianchen/2022_covid_pasc_1FL_data/20221020/main_database"

In [10]:
# general database
death_m = pd.read_csv(data_wd+"/DEATH.csv",header=0,skiprows=[1])
diagnosis_m = pd.read_csv(data_wd+"/DIAGNOSIS.csv",header=0,skiprows=[1])
encounter_m = pd.read_csv(data_wd+"/ENCOUNTER.csv",header=0,skiprows=[1])
lab_result_cm_m = pd.read_csv(data_wd+"/LAB_RESULT_CM.csv",header=0,skiprows=[1])
med_admin_m = pd.read_csv(data_wd+"/MED_ADMIN.csv",header=0,skiprows=[1])
obs_gen_m = pd.read_csv(data_wd+"/OBS_GEN.csv",header=0,skiprows=[1])
procedure_m = pd.read_csv(data_wd+"/PROCEDURES.csv",header=0,skiprows=[1])
demographics_m = pd.read_csv(data_wd+"/DEMOGRAPHIC.csv",header=0,skiprows=[1])
dispensing_m = pd.read_csv(data_wd+"/DISPENSING.csv",header=0,skiprows=[1])
immunization_m = pd.read_csv(data_wd+"/IMMUNIZATION.csv",header=0,skiprows=[1])
lds_address_history_m = pd.read_csv(data_wd+"/LDS_ADDRESS_HISTORY.csv",header=0,skiprows=[1])
obs_CLIN_m = pd.read_csv(data_wd+"/OBS_CLIN.csv",header=0,skiprows=[1])
prescribing_m = pd.read_csv(data_wd+"/PRESCRIBING.csv",header=0,skiprows=[1])
vital_m = pd.read_csv(data_wd+"/VITAL.csv",header=0,skiprows=[1])

In [11]:
print(death_m.columns == death_cov.columns)
print(diagnosis_m.columns == diagnosis_cov.columns)
print(encounter_m.columns == encounter_cov.columns)
print(lab_result_cm_m.columns == lab_result_cm_cov.columns)
print(med_admin_m.columns == med_admin_cov.columns)
print(obs_gen_m.columns == obs_gen_cov.columns)
print(procedure_m.columns == procedure_cov.columns)
print(demographics_m.columns == demographics_cov.columns)
print(dispensing_m.columns == dispensing_cov.columns)
print(immunization_m.columns == immunization_cov.columns)
print(lds_address_history_m.columns == lds_address_history_cov.columns)
print(obs_CLIN_m.columns == obs_CLIN_cov.columns)
print(prescribing_m.columns == prescribing_cov.columns)
print(vital_m.columns == vital_cov.columns)

[ True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True]
[ True  True  True  True 

In [12]:
# combine them
death = pd.concat([death_m,death_cov])
print("complete death")
diagnosis = pd.concat([diagnosis_m,diagnosis_cov])
print("complete diagnosis")
encounter = pd.concat([encounter_m,encounter_cov])
print("complete encounter")
lab_result_cm = pd.concat([lab_result_cm_m,lab_result_cm_cov])
print("complete lab_result_cm")
med_admin = pd.concat([med_admin_m,med_admin_cov])
print("complete med_admin")
obs_gen = pd.concat([obs_gen_m,obs_gen_cov])
print("complete obs_gen")
procedure = pd.concat([procedure_m,procedure_cov])
print("complete procedure")
demographics = pd.concat([demographics_m,demographics_cov])
print("complete demographics")
dispensing = pd.concat([dispensing_m,dispensing_cov])
print("complete dispensing")
immunization = pd.concat([immunization_m,immunization_cov])
print("complete immunization")
lds_address_history = pd.concat([lds_address_history_m,lds_address_history_cov])
print("complete lds_address_history")
obs_CLIN = pd.concat([obs_CLIN_m,obs_CLIN_cov])
print("complete obs_CLIN")
prescribing = pd.concat([prescribing_m,prescribing_cov])
print("complete prescribing")
vital = pd.concat([vital_m,vital_cov])
print("complete vital")

complete death
complete diagnosis
complete encounter
complete lab_result_cm
complete med_admin
complete obs_gen
complete procedure
complete demographics
complete dispensing
complete immunization
complete lds_address_history
complete obs_CLIN
complete prescribing
complete vital


In [13]:
del(death_m)
del(death_cov)
del(diagnosis_m)
del(diagnosis_cov)
del(encounter_m)
del(encounter_cov)
del(lab_result_cm_m)
del(lab_result_cm_cov)
del(med_admin_m)
del(med_admin_cov)
del(obs_gen_m)
del(obs_gen_cov)
del(procedure_m)
del(procedure_cov)
del(demographics_m)
del(demographics_cov)
del(dispensing_m)
del(dispensing_cov)
del(immunization_m)
del(immunization_cov)
del(lds_address_history_m)
del(lds_address_history_cov)
del(obs_CLIN_m)
del(obs_CLIN_cov)
del(prescribing_m)
del(prescribing_cov)
del(vital_m)
del(vital_cov)

In [14]:
death_cancer = death.loc[is_in_set_pnb(death['ID'],all_cancer_pid['ID']) ].reset_index(drop=True)
print("complete death")
diagnosis_cancer = diagnosis.loc[is_in_set_pnb(diagnosis['ID'],all_cancer_pid['ID']) ].reset_index(drop=True)
print("complete diagnosis")
encounter_cancer = encounter.loc[is_in_set_pnb(encounter['ID'],all_cancer_pid['ID']) ].reset_index(drop=True)
print("complete encounter")
lab_result_cm_cancer = lab_result_cm.loc[is_in_set_pnb(lab_result_cm['ID'],all_cancer_pid['ID']) ].reset_index(drop=True)
print("complete lab_result_cm")
med_admin_cancer = med_admin.loc[is_in_set_pnb(med_admin['ID'],all_cancer_pid['ID']) ].reset_index(drop=True)
print("complete med_admin")
obs_gen_cancer = obs_gen.loc[is_in_set_pnb(obs_gen['ID'],all_cancer_pid['ID']) ].reset_index(drop=True)
print("complete obs_gen")
procedure_cancer = procedure.loc[is_in_set_pnb(procedure['ID'],all_cancer_pid['ID']) ].reset_index(drop=True)
print("complete procedure")
demographics_cancer = demographics.loc[is_in_set_pnb(demographics['ID'],all_cancer_pid['ID']) ].reset_index(drop=True)
print("complete demographics")
dispensing_cancer = dispensing.loc[is_in_set_pnb(dispensing['ID'],all_cancer_pid['ID']) ].reset_index(drop=True)
print("complete dispensing")
immunization_cancer = immunization.loc[is_in_set_pnb(immunization['ID'],all_cancer_pid['ID']) ].reset_index(drop=True)
print("complete immunization")
lds_address_history_cancer = lds_address_history.loc[is_in_set_pnb(lds_address_history['ID'],all_cancer_pid['ID']) ].reset_index(drop=True)
print("complete lds_address_history")
obs_CLIN_cancer = obs_CLIN.loc[is_in_set_pnb(obs_CLIN['ID'],all_cancer_pid['ID']) ].reset_index(drop=True)
print("complete obs_CLIN")
prescribing_cancer = prescribing.loc[is_in_set_pnb(prescribing['ID'],all_cancer_pid['ID']) ].reset_index(drop=True)
print("complete prescribing")
vital_cancer = vital.loc[is_in_set_pnb(vital['ID'],all_cancer_pid['ID']) ].reset_index(drop=True)
print("complete vital")

complete death
complete diagnosis
complete encounter
complete lab_result_cm
complete med_admin
complete obs_gen
complete procedure
complete demographics
complete dispensing
complete immunization
complete lds_address_history
complete obs_CLIN
complete prescribing
complete vital


In [15]:
print(death_cancer.shape,len(set(death_cancer['ID'])))
print(diagnosis_cancer.shape,len(set(diagnosis_cancer['ID'])))
print(encounter_cancer.shape,len(set(encounter_cancer['ID'])))
print(lab_result_cm_cancer.shape,len(set(lab_result_cm_cancer['ID'])))
print(med_admin_cancer.shape,len(set(med_admin_cancer['ID'])))
print(obs_gen_cancer.shape,len(set(obs_gen_cancer['ID'])))
print(procedure_cancer.shape,len(set(procedure_cancer['ID'])))
print(demographics_cancer.shape,len(set(demographics_cancer['ID'])))
print(dispensing_cancer.shape,len(set(dispensing_cancer['ID'])))
print(immunization_cancer.shape,len(set(immunization_cancer['ID'])))
print(lds_address_history_cancer.shape,len(set(lds_address_history_cancer['ID'])))
print(obs_CLIN_cancer.shape,len(set(obs_CLIN_cancer['ID'])))
print(prescribing_cancer.shape,len(set(prescribing_cancer['ID'])))
print(vital_cancer.shape,len(set(vital_cancer['ID'])))

(3620, 6) 2447
(46787507, 14) 31680
(16890801, 19) 31680
(45404576, 27) 30507
(17469968, 16) 28356
(1097420, 20) 9867
(48060537, 12) 31520
(63084, 12) 31680
(4608459, 12) 19349
(160521, 20) 14644
(550209, 12) 31115
(51065316, 19) 30716
(9691091, 22) 28609
(15217281, 16) 30792


In [16]:
print(all_cancer_pid.shape,len(set(all_cancer_pid['ID'])))

(101563, 4) 31680


In [17]:
#releasing memory
del(death)
del(diagnosis)
del(encounter)
del(lab_result_cm)
del(med_admin)
del(obs_gen)
del(procedure)
del(demographics)
del(dispensing)
del(immunization)
del(lds_address_history)
del(obs_CLIN)
del(prescribing)
del(vital)

# Data quality check

## Remove duplicates

### death

In [18]:
#death
death_cancer.columns

Index(['ID', 'DEATH_DATE', 'DEATH_DATE_IMPUTE', 'DEATH_SOURCE',
       'DEATH_MATCH_CONFIDENCE', 'SOURCE_masked'],
      dtype='object')

In [19]:
print(death_cancer.shape)
print(len(set(death_cancer['ID'])))
print(len(death_cancer.loc[death_cancer['ID'].isna()]))

(3620, 6)
2447
0


In [20]:
death_cancer['DEATH_DATE'] = pd.to_datetime(death_cancer['DEATH_DATE'])
death_cancer = death_cancer.groupby('ID').agg(DEATH_DATE = ('DEATH_DATE','min'))
death_cancer = death_cancer.reset_index()

In [21]:
print(death_cancer.shape)

(2447, 2)


### demographics

In [22]:
print(demographics_cancer.shape)
print(len(set(demographics_cancer['ID'])))
print(len(demographics_cancer.loc[demographics_cancer['BIRTH_DATE'].isna()]))

(63084, 12)
31680
0


In [23]:
demographics_cancer = demographics_cancer.drop_duplicates('ID').reset_index()

In [24]:
print(demographics_cancer.shape)

(31680, 13)


### other duplicates to be investigated

In [4]:
death_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/death_cancer.pkl','rb'))
diagnosis_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/diagnosis_cancer.pkl','rb'))
encounter_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/encounter_cancer.pkl','rb'))
lab_result_cm_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/lab_result_cm_cancer.pkl','rb'))
med_admin_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/med_admin_cancer.pkl','rb'))
obs_gen_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/obs_gen_cancer.pkl','rb'))
procedure_cancer=pickle.load( open('/data/datasets/changyuyin/2_COVIDwithCancer/procedure_cancer.pkl','rb'))
demographics_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/demographics_cancer.pkl','rb'))
dispensing_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/dispensing_cancer.pkl','rb'))
immunization_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/immunization_cancer.pkl','rb'))
lds_address_history_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/lds_address_history_cancer.pkl','rb'))
obs_CLIN_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/obs_CLIN_cancer.pkl','rb'))
prescribing_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/prescribing_cancer.pkl','rb'))
vital_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/vital_cancer.pkl','rb'))

In [5]:
death_cancer=death_cancer.drop_duplicates(ignore_index=True)
diagnosis_cancer=diagnosis_cancer.drop_duplicates(ignore_index=True)
encounter_cancer=encounter_cancer.drop_duplicates(ignore_index=True)
lab_result_cm_cancer=lab_result_cm_cancer.drop_duplicates(ignore_index=True)
med_admin_cancer=med_admin_cancer.drop_duplicates(ignore_index=True)
obs_gen_cancer=obs_gen_cancer.drop_duplicates(ignore_index=True)
procedure_cancer=procedure_cancer.drop_duplicates(ignore_index=True)
demographics_cancer=demographics_cancer.drop_duplicates(ignore_index=True)
dispensing_cancer=dispensing_cancer.drop_duplicates(ignore_index=True)
immunization_cancer=immunization_cancer.drop_duplicates(ignore_index=True)
lds_address_history_cancer=lds_address_history_cancer.drop_duplicates(ignore_index=True)
obs_CLIN_cancer=obs_CLIN_cancer.drop_duplicates(ignore_index=True)
prescribing_cancer=prescribing_cancer.drop_duplicates(ignore_index=True)
vital_cancer=vital_cancer.drop_duplicates(ignore_index=True)

# write 

In [6]:
pickle.dump(death_cancer, open('/data/datasets/changyuyin/2_COVIDwithCancer/death_cancer.pkl','wb'))
pickle.dump(diagnosis_cancer, open('/data/datasets/changyuyin/2_COVIDwithCancer/diagnosis_cancer.pkl','wb'))
pickle.dump(encounter_cancer, open('/data/datasets/changyuyin/2_COVIDwithCancer/encounter_cancer.pkl','wb'))
pickle.dump(lab_result_cm_cancer, open('/data/datasets/changyuyin/2_COVIDwithCancer/lab_result_cm_cancer.pkl','wb'))
pickle.dump(med_admin_cancer, open('/data/datasets/changyuyin/2_COVIDwithCancer/med_admin_cancer.pkl','wb'))
pickle.dump(obs_gen_cancer, open('/data/datasets/changyuyin/2_COVIDwithCancer/obs_gen_cancer.pkl','wb'))
pickle.dump(procedure_cancer, open('/data/datasets/changyuyin/2_COVIDwithCancer/procedure_cancer.pkl','wb'))
pickle.dump(demographics_cancer, open('/data/datasets/changyuyin/2_COVIDwithCancer/demographics_cancer.pkl','wb'))
pickle.dump(dispensing_cancer, open('/data/datasets/changyuyin/2_COVIDwithCancer/dispensing_cancer.pkl','wb'))
pickle.dump(immunization_cancer, open('/data/datasets/changyuyin/2_COVIDwithCancer/immunization_cancer.pkl','wb'))
pickle.dump(lds_address_history_cancer, open('/data/datasets/changyuyin/2_COVIDwithCancer/lds_address_history_cancer.pkl','wb'))
pickle.dump(obs_CLIN_cancer, open('/data/datasets/changyuyin/2_COVIDwithCancer/obs_CLIN_cancer.pkl','wb'))
pickle.dump(prescribing_cancer, open('/data/datasets/changyuyin/2_COVIDwithCancer/prescribing_cancer.pkl','wb'))
pickle.dump(vital_cancer, open('/data/datasets/changyuyin/2_COVIDwithCancer/vital_cancer.pkl','wb'))